# Monotone

1. All cells of a region must have the same color.

2. All black cells must form an orthogonally continuous area.

3. No 2 x 2 cell area within the grid can have the same color.

----

1. 一个区域的所有单元格必须具有相同的颜色。
2. 网格内的 2 x 2 像元区域不能具有相同的颜色。
3. 所有黑细胞必须形成一个正交连续的区域。


In [1]:

import numpy as np
from ortools.sat.python import cp_model as cp
import gurobipy as gp
from collections import deque
from itertools import chain

In [2]:
def readGrid(path):
    with open(f"../assets/data/monotone/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("10x10_1")
    for g in grid:
        print(g)

['A', 'B', 'B', 'C', 'C', 'C', 'D', 'E', 'F', 'G']
['A', 'H', 'I', 'J', 'C', 'K', 'D', 'F', 'F', 'F']
['A', 'A', 'I', 'J', 'L', 'L', 'M', 'N', 'O', 'P']
['Q', 'R', 'I', 'I', 'S', 'S', 'N', 'N', 'P', 'P']
['Q', 'Q', 'T', 'T', 'U', 'U', 'V', 'V', 'P', 'W']
['X', 'Q', 'Y', 'T', 'U', 'Z', 'a', 'a', 'b', 'W']
['c', 'c', 'D', 'e', 'e', 'e', 'f', 'f', 'b', 'b']
['g', 'c', 'h', 'h', 'I', 'e', 'j', 'f', 'k', 'l']
['g', 'g', 'm', 'n', 'o', 'o', 'p', 'p', 'k', 'k']
['g', 'm', 'm', 'n', 'n', 'o', 'o', 'p', 'p', 'k']


In [16]:
def MonotoneSolver(m, n, grid):
    Monotone = gp.Model("Monotone")

    Monotone.modelSense = gp.GRB.MAXIMIZE
    Monotone.Params.lazyConstraints = 1
    Monotone.update()
    cells = {}
    x = {}
    for i in range(m):
        for j in range(n):
            x[i, j] = Monotone.addVar(
                vtype = gp.GRB.BINARY,
                obj = 1,
                name = f"x[{i},{j}]")
            if grid[i][j] not in cells:
                cells[grid[i][j]] = [(i, j)]
            else:
                cells[grid[i][j]].append((i, j))
    
    for k, v in cells.items():
        if len(v) > 1:
            for i in range(len(v) - 1):
                Monotone.addConstr(x[v[i][0], v[i][1]] == x[v[i + 1][0], v[i + 1][1]], name = f"equal_{k}_{i}")
        
    
    for i in range(m - 1):
        for j in range(n - 1):
            Monotone.addConstr(gp.quicksum([
                x[i, j], x[i + 1, j], x[i, j + 1], x[i + 1, j + 1]
            ]) >= 1, name = f">=1_{i}_{j}")
            Monotone.addConstr(gp.quicksum([
                x[i, j], x[i + 1, j], x[i, j + 1], x[i + 1, j + 1]
            ]) <= 3, name = f"<=3_{i}_{j}")
    
    Monotone.setObjective(gp.quicksum(x[i, j] for i in range(m) for j in range(n)), gp.GRB.MAXIMIZE)
    # Monotone.write("Monotone.lp")
    def border_elim(model, where):
        if (where == gp.GRB.Callback.MIPSOL):
            
            x_sol = model.cbGetSolution(model._x)
            curr_grid = [[0] * n for i in range(m)]
            for i in range(m):
                for j in range(n):
                    if x_sol[i, j] > 1e-6:
                        curr_grid[i][j] = 1
                        
            # for g in curr_grid:
            #     print(g)
                        
            def find_surrounding_black_cells(grid):
                rows, cols = len(grid), len(grid[0])
                visited = set()
                directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, down, left, right movements
                total_white = sum(chain.from_iterable(grid))
                # Find the first unvisited white cell
                def find_unvisited_white():
                    for i in range(rows):
                        for j in range(cols):
                            if grid[i][j] == 1 and (i, j) not in visited:
                                return (i, j)
                    return None
                borders_cut = []
                cnt_white = 0
                while len(visited) < total_white:
                    start = find_unvisited_white()
                    if not start:
                        return []

                    # Use BFS to find all connected white cells and record the surrounding black cells
                    queue = deque([start])
                    surrounding_black_cells = set()  # Use a set to avoid duplicates
                    
                    while queue:
                        x, y = queue.popleft()
                        if (x, y) in visited:
                            continue
                        visited.add((x, y))
                        cnt_white += 1
                        # Check all four adjacent directions
                        for dx, dy in directions:
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < rows and 0 <= ny < cols:
                                if grid[nx][ny] == 1 and (nx, ny) not in visited:
                                    queue.append((nx, ny))
                                elif grid[nx][ny] == 0:
                                    surrounding_black_cells.add((nx, ny))
                    borders_cut.append(list(surrounding_black_cells))
                if cnt_white == sum(chain.from_iterable(grid)) and len(borders_cut) == 1:
                    return []
                borders_cut.sort(key=lambda x: len(x))
                return borders_cut

            borders_cut = find_surrounding_black_cells(curr_grid)
            for cut in borders_cut:
                print("ADD Cut")
                # print(cut)
                model.cbLazy(gp.quicksum(x[subx, suby] for (subx, suby) in cut ) >= 1)
    
    Monotone._x = x
    Monotone.optimize(border_elim)
    
    for i in range(m):
        for j in range(n):
            if x[i, j].x > 1e-6:
                print("X", end = " ")
            else:
                print("0", end = " ")
        print()
    print()


if __name__ == "__main__":
    m, n, grid = readGrid("10x10_1")
    MonotoneSolver(m, n, grid)
    

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 220 rows, 100 columns and 764 nonzeros
Model fingerprint: 0x08a22693
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
50
Found heuristic solution: objective 50.0000000
Presolve removed 206 rows and 88 columns
Presolve time: 0.00s
Presolved: 14 rows, 12 columns, 44 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
63
Found heuristic solution: objective 63.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 63 50 

Optimal s